In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

import numpy as np
import torch
from torch.nn import functional as F
import torch.cuda.amp as amp
import matplotlib.pyplot as plt
import IPython.display as ipd
import wandb

from deepspeech import model, train, datasets, utils, predict

## Prediction

In [ ]:
m, pt = predict.load('purzelrakete/feldberlin-deepspeech/212d0t74')
p = m.cfg
p.graphemes = np.array(p.graphemes)

In [ ]:
ds = datasets.yesno(p)
ds = datasets.SpecAugmented(ds, p, masked=False)

In [ ]:
batch = [ds[i] for i in range(2)]
x, xn, y, yn = datasets.batch(p)(batch)

In [ ]:
yhat = predict.predict(m, x, xn)
yhat = predict.decode_argmax(yhat, p)

In [ ]:
yhat